## Final Project Submission

Please fill out:
* Student name: Tabitha Berum
* Student pace: full time
* Scheduled project review date/time: 22, March 2024
* Instructor name: Diana Mongina
* Blog post URL: N/A


# Overview
The project's goal is to create an analysis using information on movies, including production budgets, sales and the percentage of viewers that like each film, gathered from many websites. This analysis's outcome will make it easier to select the greatest film to produce financially for the future Microsoft Studios.

# Business Understanding
Microsoft Studios wants to produce original video content, but they have no idea how to go about doing so. The goal of this analysis is to explore what types of films are doing the best and will most likely generate revenue post production.
This will be done in the following steps:
    1. Defining key factors to evaluate a film's profitability using production budget, ratings or gross revenue.
    2. Examining profitability of different movie attributes like release date, genre, director, length of the movie and  popularity.
    3. Coming up with a step by step guide of choosing the best movie financially using the 5 W's to gether with the attributes mentioned above.

# Data Understanding


##      Data Preparation

In [18]:
# Import standard libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [19]:
# Import library that enables us to read zip files
import zipfile 

#Extract IMDB SQL .db file
# with zipfile.ZipFile('./zippedData/im.db.zip') as zipObj:

#    zipObj.extractall('./zippedData/')

In [23]:
#Read tables using sqlite3
import sqlite3
conn = sqlite3.connect('ZippedData/im.db')

In [21]:
# Loading data from all datasets
movie_data = pd.read_csv('./zippeddata/rt.movie_info.tsv.gz', sep="\t")
reviews_data = pd.read_csv('./zippedData/rt.reviews.tsv.gz', sep="\t", encoding= 'unicode_escape')
gross_data = pd.read_csv('./zippedData/bom.movie_gross.csv.gz')
tmdb_data = pd.read_csv('./zippedData/tmdb.movies.csv.gz')
budgets_data = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz')



After loading the data , we are going to explore each dataset to know if there's any anomalies. We begin with the Rotten Tomatoes dataset

In [22]:
#